In [ ]:
import copy
from typing import Any, List

from pydantic import BaseModel

import autogen
from autogen.agentchat.conversable_agent import ConversableAgent
from autogen.agents.experimental import WebSurferAgent

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-4o-mini"]},
)
llm_config = {"config_list": config_list}

answer_question("What are the top most popular posts on hackernews?", llm_config)

In [ ]:
def answer_question(
    question: str,
    llm_config: dict[str, Any],
    max_web_steps: int = 3,
) -> str:
    class InformationCrumb(BaseModel):
        source_url: str
        source_title: str
        source_summary: str
        relevant_info: str

    class GatheredInformation(BaseModel):
        information: List[InformationCrumb]

        def format(self) -> str:
            return "Here is the gathered information: \n" + "\n".join(
                f"URL: {info.source_url}\nTitle: {info.source_title}\nSummary: {info.source_summary}\nRelevant Information: {info.relevant_info}\n\n"
                for info in self.information
            )

    websurfer_config = copy.deepcopy(llm_config)

    websurfer_config["config_list"][0]["response_format"] = GatheredInformation

    def is_termination_msg(x):
        return x.get("content", "") and x.get("content", "").startswith("Answer confirmed:")

    websurfer = WebSurferAgent(
        llm_config=websurfer_config,
        name="WebSurferAgent",
        system_message=(
            "You are a web surfer agent responsible for gathering information from the web to provide information for answering a question\n"
            "You will be asked to find information related to the question and provide a summary of the information gathered.\n"
            "The summary should include the URL, title, summary, and relevant information for each piece of information gathered.\n"
        ),
        is_termination_msg=is_termination_msg,
        human_input_mode="NEVER",
        web_tool_kwargs={"agent_kwargs": {"max_steps": max_web_steps}},
    )

    websurfer_critic = ConversableAgent(
        name="WebSurferCritic",
        system_message=(
            "You are a critic agent responsible for evaluating the answer provided by the web surfer agent.\n"
            "You need to confirm wehther the information provided by the websurfer is correct and sufficient to answer the question.\n"
            "You can ask the web surfer to provide more information or provide and confirm the answer.\n"
        ),
        llm_config=llm_config,
        is_termination_msg=is_termination_msg,
        human_input_mode="NEVER",
    )

    @websurfer.register_for_execution()
    @websurfer_critic.register_for_llm(
        description="Call this method when you agree that the original question can be answered with the gathered information and provide the answer."
    )
    def confirm_answer(answer: str) -> str:
        return "Answer confirmed: " + answer

    websurfer_critic.register_for_execution()(websurfer.tool)

    result = websurfer_critic.initiate_chat(
        websurfer,
        message="Please find the answer to this question: " + question,
    )

    return result.summary


answer_question("What are the top most popular posts on hackernews?", llm_config)